### Currently pseudo code showing what functions we need
Note "allTickers" should be read as loop through all tickers

In [ ]:
#The essentials
import pandas as pd
from dotenv import load_dotenv
load_dotenv()

tickers = pd.read_csv('./Data/Cleaned_Data/Ticker_library.csv')['Ticker'].to_list()

In [ ]:
import libs.krista_lib as news
# News Handling
News_allTickers = [news.form_df(ticker) for ticker in tickers] #or read already created csv
#Append list into one DF should be ['Ticker','Date','Text']

In [ ]:
import libs.reddit as red
# Reddit Handling
Reddit_allTickers = [red.pull_all_about(ticker,from_subreddits=['subredditList']) for ticker in tickers] #or read already created csv
#Append list into one DF should be ['BusinessMentioned','Date','Text']

In [ ]:
import libs.sec as sec
# SEC Handling
SEC_allTickers = pd.read_csv('.\Data\Cleaned_Data\sec_df.csv') #it is definately too big to do on the fly
#DF should be ['Ticker','Date','Text']

In [ ]:
# Precessing (uses binning, vadar, and tokenizer code)
import libs.df_helper as dfh
News_vadar = dfh.vadar_time_Bin(News_allTickers,days=7) # Returns a df with headers ['Ticker','time bin','avg vader score of all in bin']
Reddit_vadar = dfh.vadar_time_Bin(Reddit_allTickers,days=7) # Returns a df with headers ['Ticker','time bin','avg vader score of all in bin']
SEC_vadar = dfh.vadar_time_Bin(SEC_allTickers,days=45) # Returns a df with headers ['Ticker','time bin','avg vader score of all in bin']

working_DF = dfh.combiner(News_vadar,Reddit_vadar,SEC_vadar) # Returns a dataframe where the dates match (copies SECs larger bins over every cell in its date range)
#Add in actual stock data

In [ ]:
# plotting to see trends

In [ ]:
# Splitting
    # Train Test split for Neural Net
X_train={}
y_train={}
X_test={}
y_test={}
for ticker in tickers:
    X_train[ticker]=None #TODO
    y_train[ticker]=None #TODO
    X_test[ticker]=None #TODO
    y_test[ticker]=None #TODO

In [ ]:
# Neural Net
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

for ticker in tickers:
    # Define the LSTM RNN model.
    model = Sequential()

    # Initial model setup
    number_units = 30
    dropout_fraction = 0.2

    # Layer 1
    model.add(LSTM(
        units=number_units,
        return_sequences=True,
        input_shape=(X_train[ticker].shape[1], 1))
        )
    model.add(Dropout(dropout_fraction))

    # Layer 2
    model.add(LSTM(units=number_units, return_sequences=True))
    model.add(Dropout(dropout_fraction))

    # Layer 3
    model.add(LSTM(units=number_units))
    model.add(Dropout(dropout_fraction))

    # Output layer
    model.add(Dense(1))

    # Compile the model
    model.compile(optimizer="adam", loss="mean_squared_error")

    # Train the model
    model.fit(X_train[ticker], y_train[ticker], epochs=10, shuffle=False, batch_size=90, verbose=1)

    # Evaluate the model
    model.evaluate(X_test[ticker], y_test[ticker], verbose=0)

    # Make predictions using the testing data X_test
    predicted = model.predict(X_test[ticker])

In [ ]:
# Signal Generation
signals_df = {}
for ticker in tickers:
    signals_df[ticker]['Signal'] = None # TODO

In [ ]:
# Back Testing
for ticker in tickers:
    # Set initial capital
    initial_capital = float(100000)

    # Set the share size
    share_size = 500

    # Buy a 500 share position when the dual moving average crossover Signal equals 1
    # Otherwise, `Position` should be zero (sell)
    signals_df[ticker]['Position'] = share_size * signals_df[ticker]['Signal']

    # Determine the points in time where a 500 share position is bought or sold
    signals_df[ticker]['Entry/Exit Position'] = signals_df[ticker]['Position'].diff()

    # Multiply the close price by the number of shares held, or the Position
    signals_df[ticker]['Portfolio Holdings'] = signals_df[ticker]['close'] * signals_df[ticker]['Position']

    # Subtract the amount of either the cost or proceeds of the trade from the initial capital invested
    signals_df[ticker]['Portfolio Cash'] = initial_capital - (signals_df[ticker]['close'] * signals_df[ticker]['Entry/Exit Position']).cumsum() 

    # Calculate the total portfolio value by adding the portfolio cash to the portfolio holdings (or investments)
    signals_df[ticker]['Portfolio Total'] = signals_df[ticker]['Portfolio Cash'] + signals_df[ticker]['Portfolio Holdings']

    # Calculate the portfolio daily returns
    signals_df[ticker]['Portfolio Daily Returns'] = signals_df[ticker]['Portfolio Total'].pct_change()

    # Calculate the portfolio cumulative returns
    signals_df[ticker]['Portfolio Cumulative Returns'] = (1 + signals_df[ticker]['Portfolio Daily Returns']).cumprod() - 1

In [ ]:
# Visualize the Backtest

# Visualize exit position relative to total portfolio value
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Portfolio Total'].hvplot.scatter(
    color='yellow',
    marker='v',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize entry position relative to total portfolio value
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Portfolio Total'].hvplot.scatter(
    color='purple',
    marker='^',
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize the value of the total portfolio
total_portfolio_value = signals_df[['Portfolio Total']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    xlabel='Date',
    width=1000,
    height=400
)

# Overlay the plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(
    title="Apple Algorithm - Total Portfolio Value",
    yformatter='%.0f'
)
